# Writing exercises

The scicode-widgets mainly offers a flexible code widget that allows instant feedback to evaluate the code for interactive plots.

In [ ]:
from scwidgets import (
    CodeExercise,
    CodeInput,
    CueFigure,
    CueOutput,
    CueObject,
    ExerciseRegistry,
    ParametersPanel,
    TextExercise,
)

from ipywidgets import FloatSlider, IntSlider
import matplotlib.pyplot as plt
import numpy as np

## ExerciseRegistry to store and load answers to exercises

Due to limitations of jupyter notebooks in saving the widgets state, we store and load the widget state by ourself using the `ExerciseRegstry`. It allows you specify a filename for the json file to store all the answers to all registered exercises. The exercises are registered by passing the `ExerciseRegstry` instance as input argument.

In [ ]:
exercise_registry = ExerciseRegistry()
exercise_registry

## TextExercise to create text exercises

A simple textarea to save students answers. Note that the save and load buttons only appear when a exercise key and registry is given

In [ ]:
TextExercise(
    title="Exercise 01: Derive a solution for weights",
    description="""
    We can define ridge regression by extending the ordinary least
    square solution by a penalization $\lambda\|\mathbf{w}\|_2^2$. Please derive the solution
    for the weights from the optimization problem:</p>
    $$\hat{\mathbf{w}} = \min_\mathbf{w} \|\mathbf{y}-\mathbf{X}\mathbf{w}\|^2 + \lambda\|\mathbf{w}\|^2$$""",
    key="ex01", # the key it is stored under in the json file
    exercise_registry=exercise_registry
)

## Interactive coding exercises

This is an example how to create a simple exercise.

In [ ]:
# This is what the students sees and can adapt
def sin(x: int, omega=1.0):
    """
    Implements ridge regression

    :param x: An array of data points
    :param omega: The frequency
    """
    import numpy as np
    return np.sin(x*omega)

def update_func(code_ex):
    x = np.linspace(-2*np.pi, 2*np.pi, 100)
    y = code_ex.code(x, code_ex.parameters["omega"])
    ax = code_ex.figure.gca()
    ax.plot(x, y)
    
code_ex_description = """
Implements a sinus function $\sin(x\omega)$.
"""
code_ex = CodeExercise(
    code=sin,
    parameters={'omega': (0.5, 3.14, 0.1)},
    outputs=plt.figure(),
    update=update_func,
    update_mode="continuous", # we also support ["manual", "release"]
    title="Sinus function",
    description=code_ex_description,
    key="sin_local",
    exercise_registry=exercise_registry
)

code_ex.run_update() # For the demonstration we run the widget one time
display(code_ex)

### Creating widget components beforehand

More complex widgets might need creation beforehand to allow full customization

In [ ]:
from ipywidgets import HTML

# One can pass a function also by
code_input = CodeInput(
    function_name="sin",
    function_parameters="x: int, omega=1.0",
    function_body="import numpy as np\nreturn np.sin(x*omega)",
    docstring="Implements ridge regression\n\n:param x: An array of data points\n:param omega: The frequency"
)
# customization of figure toolbar, only important in widget mode (use %matplotlib widget)
figure = CueFigure(plt.figure(), show_toolbars=True)
# to use a custom output for own widgets
output = CueOutput()
# to use display custom widgets
table = CueObject(HTML(value="<table><tr><th>x</th><th>y</th></tr></table>"))


# to customize sliders, one can directy
parameter_panel = ParametersPanel(
    omega=FloatSlider(value=1, min=0.5, max=3.14, step=0.1, description="$\\omega$")
)
# alternatively if passed to CodeExercise this also works
#parameter_panel = dict(
#    omega=FloatSlider(value=1, min=0.5, max=3.14, step=0.1, description="$\\omega$")
#)


def update_func(code_ex):
    x = np.linspace(-2*np.pi, 2*np.pi, 100)
    y = code_ex.code(x, code_ex.parameters["omega"])
    ax = code_ex.outputs[0].figure.gca()
    ax.plot(x, y)
    with code_ex.outputs[1]:
        print("Some text after the figure")

    code_ex.outputs[2].object.value = "<table style=\"width:50%\"><tr><th>x</th><th>y</th></tr>" + \
         "".join([f"<tr><td>{x[i]:.2f}</td><td>{y[i]:.2f}</td></tr>" for i in range(0, len(x), 20)]) + \
         "</table>"
    # the captured text in the function is always printed before any other output
    print("Some text before the figure")
    
    

code_ex = CodeExercise(
    code=code_input,
    parameters=parameter_panel,
    outputs=[figure, output, table],
    update=update_func,
)

code_ex.run_update() # For the demonstration we run the widget one time
display(code_ex)

### Include imports to code input

So far we always added the imports required for the code inside the code input. We need to do this because the widget is its creates its own environment (own globals) so no function from the notebook is accidently used. This however does not solve the problem when one wants to include typehints in the function signature that require imports. For that one can add the library to the globals.

In [ ]:
def sin(x: np.ndarray, omega=1.0): # using np.ndarray requires import numpy before the function body
    """
    Implements ridge regression

    :param x: An array of data points
    :param omega: The frequency
    """
    return np.sin(x*omega)

def update_func(code_ex):
    x = np.linspace(-2*np.pi, 2*np.pi, 100)
    y = code_ex.code(x, code_ex.parameters["omega"])
    ax = code_ex.figure.gca()
    ax.plot(x, y)
    
code_ex = CodeExercise(
    code=CodeInput(sin, builtins={'np': np}),
    parameters={'omega': (0.5, 3.14, 0.1)},
    outputs=plt.figure(),
    update=update_func,
)

code_ex.run_update() # For the demonstration we run the widget one time
display(code_ex)

### Interactive coding exercises with globals variables

This is an example how to create a simple exercise using globals in the update function. This can be more convenient in certain cases but a bit more prone to errors as when creating multiple exercises the global names can easily conflict with each other and result in unwanted behavior. Therefore we recommend use the code demo instance that is passed through the update function.

In [ ]:
# This is what the students sees and can adapt
def sin(x, omega):
    """
    Implements ridge regression

    :param x: An array of data points
    :param omega: The frequency
    """
    import numpy as np
    return np.sin(x*omega)

code_input = CodeInput(sin)
cue_figure = CueFigure(plt.figure())
parameter_panel = ParametersPanel(
    omega=FloatSlider(value=1, min=0.5, max=3.14, step=0.1, description="$\\omega$")
)

x = np.linspace(-2*np.pi, 2*np.pi, 100)
def update_func():
    global x, code_input, cue_figure, parameter_panel    
    y = code_ex.code(x, parameter_panel.parameters["omega"])
    ax = cue_figure.figure.gca()
    ax.plot(x, y)
    
    
code_ex = CodeExercise(
    code=code_input,
    parameters=parameter_panel,
    outputs=cue_figure,
    update=update_func,
)


code_ex.run_update() # For the demonstration we run the widget one time
display(code_ex)

## ParametersPanel short constructors

The `ParametersPanel` can be also used with the same shorthand constructors as [interact](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html). Here some examples

In [ ]:
from ipywidgets import fixed
ParametersPanel(
    frequency=(0.5, 2*np.pi, 0.1),
    amplitude=(1, 5, 1),
    inverted=True,
    type=["sin", "cos"],
    plot_title="trigonometric curve",
    const=fixed(1) # this argument will be passed but is not changeable and therefore not displayed
)